In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


In [2]:
## The goal of this notebook is to have some decomposition of the sales vector x as A x, for some matrix A.

## Looking at both what we can reconstruct empirically and hat we can make up in simple models of circulation.


In [7]:
df = pd.read_csv('./../../data/processed/PYOD_FY.csv')
df = df.loc[df.YEAR > 2007]

# df.PYOD = df.PYOD.replace('XU', 'RW')

In [8]:
df_ = df.copy()

## Next step is to reduce the number of countries.
x = df_.groupby('PYOD')['VART'].sum().sort_values(ascending = False)
main_countries = (100*x/x.sum()).round(3).head(30)

c = main_countries.index
df_['C'] = df_['PYOD'].map(dict(zip(c, c))).fillna('RW')

In [9]:
df_ = df_.groupby(['ID', 'C', 'IMPORT', 'YEAR'])['VART'].sum().reset_index()

In [4]:
## now, I group the French firms in 10 random groups of equal weight.

In [10]:
sales = df_.groupby('ID')['VART'].sum()
sales_ = sales.sample(frac=1)

In [11]:
P = 10
parts = pd.cut(sales_.cumsum()/sales_.sum(), P, labels = range(P)).sort_index()
parts.name = 'P'

In [ ]:
## Now, merge and groupby these parts. 
df_ = df_.merge(parts.reset_index())

In [13]:
df_ = df_.groupby(['P', 'C', 'IMPORT', 'YEAR'])['VART'].sum().reset_index()

In [14]:
df_['i'] = np.nan
df_['j'] = np.nan

df_.loc[df_.IMPORT == 0, 'i'] = df_.loc[df_.IMPORT == 0, 'P']
df_.loc[df_.IMPORT == 1, 'i'] = df_.loc[df_.IMPORT == 1, 'C']

df_.loc[df_.IMPORT == 0, 'j'] = df_.loc[df_.IMPORT == 0, 'C']
df_.loc[df_.IMPORT == 1, 'j'] = df_.loc[df_.IMPORT == 1, 'P']


In [15]:
pd.options.display.max_columns = 99

In [ ]:
## World trade

In [37]:
wt = pd.read_csv('./../../data/wt/year_origin_destination_hs07_4.tsv', '\t', usecols=['year','origin','dest','export_val','import_val'])
wt = wt.loc[wt.year < 2014]
wt = wt.groupby(['year', 'origin', 'dest'])[['export_val', 'import_val']].sum().reset_index()
wt[['export_val', 'import_val']] = wt[['export_val', 'import_val']]/1.3

In [50]:
cntry_data = pd.read_csv('./c_table.csv').rename(columns = {'ISO3166-1-Alpha-2': 'iso2'})
ccodes = cntry_data[['ccode', 'iso2']]

In [83]:
# exclude france
wt_ = wt.loc[(wt.origin != 'fra') & (wt.dest != 'fra')]

In [84]:
wt_ = wt_.merge(ccodes, left_on = 'origin', right_on = 'ccode').merge(ccodes, left_on = 'dest', right_on = 'ccode')

wt_['i'] = wt_['iso2_x'].map(dict(zip(c, c))).fillna('RW')
wt_['j'] = wt_['iso2_y'].map(dict(zip(c, c))).fillna('RW')

# Info in imports column is redundant, fixed to transpose of exports.
wt_ = wt_.groupby(['i', 'j', 'year'])[['export_val']].sum().reset_index()

In [234]:
out = []
for year in range(2008, 2014):
    french_part = df_.loc[df_.YEAR == year].set_index(['i', 'j'])['VART'].unstack().round(-7)/1e9
    intnl_part = wt_.loc[wt_.year == year].set_index(['i', 'j'])['export_val'].unstack().round(-7)/1e9
    intnl_part = intnl_part.reindex(index = french_part.index, columns=french_part.columns)
    
    full_matrix = french_part.fillna(intnl_part)
    
    # Modelling unobserved
    # Size of internal circulation: (should be about 5 times exports)
    C = 5

    for k in range(P, len(full_matrix)):
        full_matrix.iloc[k, k] = full_matrix.fillna(0).iloc[k, k] + 5 * full_matrix.sum(1).iloc[k]

    fr_fill = np.round(C*full_matrix.iloc[:P].sum().sum()/(P**2), 2)

    full_matrix.iloc[:P, :P] = fr_fill

    full_matrix = full_matrix.fillna(0)

    out += [full_matrix]
    
    # values are fairly stable
    # pd.concat([table.stack() for table in out], axis = 1).sample(20)

In [264]:

A = out[3].T.div(out[3].sum(1), axis = 0)
At = out[3].div(out[3].sum(1), axis = 1)
s = out[3].sum(1)

# At.dot(s) = s.dot(A) = s

## Variaran mucho los elementos de At en el timepo??

At_list = [table.div(table.sum(1), axis = 1).stack() for table in out]

In [207]:
100*pd.concat(At_list, axis = 1).sample(20).round(5)

,,0,1,2,3,4,5
i,j,,,,,,
1,BR,0.129,0.121,0.178,0.177,0.180,0.216
CA,3,0.014,0.014,0.013,0.021,0.023,0.011
NO,DE,2.452,2.328,2.020,2.012,2.264,3.176
6,US,0.525,0.570,0.523,0.515,0.546,0.599
ES,JP,0.150,0.171,0.169,0.190,0.212,0.206
PT,0,0.173,0.197,0.205,0.193,0.192,0.204
CH,NO,0.081,0.070,0.060,0.060,0.077,0.057
PL,US,0.287,0.348,0.352,0.400,0.432,0.420
RU,TR,0.702,0.663,0.649,0.489,0.541,0.502


In [275]:
x = At_list[2].unstack()
At_n2 = pd.DataFrame(np.linalg.matrix_power(x, 200), index=x.index, columns=x.columns)

In [259]:
import seaborn as sns
# np.random.seed(25)
# cmap = cmap=sns.diverging_palette(5, 250, as_cmap=True)
# bigdf = pd.DataFrame(.T)

cm = sns.light_palette("green", as_cmap=True)

# s = full_matrix.fillna(0).style.background_gradient(cmap=cm)
# s

In [272]:
At.dot(s) - s

i
0    -5.684342e-14
1    -5.684342e-14
2     0.000000e+00
3    -5.684342e-14
4    -5.684342e-14
5    -1.136868e-13
6    -5.684342e-14
7    -2.842171e-14
8    -2.842171e-14
9     0.000000e+00
AT    3.410605e-13
BR    0.000000e+00
CA    2.273737e-13
CH   -2.273737e-13
CN    3.637979e-12
CZ   -2.273737e-13
DE    1.818989e-12
DK    0.000000e+00
DZ    1.136868e-13
ES    0.000000e+00
GB    0.000000e+00
HU    1.136868e-13
IE   -1.136868e-13
IN    2.273737e-13
IT   -4.547474e-13
JP    4.547474e-13
KR    9.094947e-13
MA    0.000000e+00
NL    0.000000e+00
NO    1.136868e-13
PL    0.000000e+00
PT    0.000000e+00
RU    0.000000e+00
RW    3.637979e-12
SA    2.273737e-13
SE    1.136868e-13
SG    0.000000e+00
TN    0.000000e+00
TR    0.000000e+00
US    9.094947e-13
XU    0.000000e+00
dtype: float64

In [282]:
pd.DataFrame(np.linalg.inv(np.eye(len(At)) - At))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40
0,4.865608e+15,4.705436e+15,4.748699e+15,4.909665e+15,4.681131e+15,4.714213e+15,4.864877e+15,4.807598e+15,4.929341e+15,5.066791e+15,4.027113e+15,3.425324e+15,4.588474e+15,3.519408e+15,2.435829e+15,3.476291e+15,3.289714e+15,3.438716e+15,2.912375e+15,5.124695e+15,5.489584e+15,3.389866e+15,2.199302e+15,5.675476e+15,4.190016e+15,3.392631e+15,3.225405e+15,7.142991e+15,4.252854e+15,2.200687e+15,4.288847e+15,5.894802e+15,2.345175e+15,3.888268e+15,1.726310e+15,3.412077e+15,4.532960e+15,5.360763e+15,5.691463e+15,5.605705e+15,4.567204e+15
1,4.956476e+15,4.793312e+15,4.837383e+15,5.001355e+15,4.768554e+15,4.802253e+15,4.955732e+15,4.897383e+15,5.021399e+15,5.161416e+15,4.102322e+15,3.489293e+15,4.674166e+15,3.585135e+15,2.481320e+15,3.541212e+15,3.351151e+15,3.502936e+15,2.966765e+15,5.220401e+15,5.592105e+15,3.453173e+15,2.240375e+15,5.781469e+15,4.268267e+15,3.455991e+15,3.285642e+15,7.276390e+15,4.332279e+15,2.241786e+15,4.368943e+15,6.004890e+15,2.388972e+15,3.960884e+15,1.758550e+15,3.475799e+15,4.617615e+15,5.460878e+15,5.797754e+15,5.710394e+15,4.652499e+15
2,4.926122e+15,4.763957e+15,4.807758e+15,4.970726e+15,4.739351e+15,4.772843e+15,4.925382e+15,4.867390e+15,4.990647e+15,5.129807e+15,4.077199e+15,3.467924e+15,4.645541e+15,3.563179e+15,2.466124e+15,3.519525e+15,3.330628e+15,3.481484e+15,2.948596e+15,5.188431e+15,5.557858e+15,3.432026e+15,2.226655e+15,5.746062e+15,4.242128e+15,3.434826e+15,3.265520e+15,7.231828e+15,4.305747e+15,2.228057e+15,4.342187e+15,5.968115e+15,2.374342e+15,3.936627e+15,1.747780e+15,3.454513e+15,4.589336e+15,5.427435e+15,5.762248e+15,5.675423e+15,4.624006e+15
3,4.796923e+15,4.639011e+15,4.681663e+15,4.840357e+15,4.615050e+15,4.647664e+15,4.796202e+15,4.739731e+15,4.859755e+15,4.995265e+15,3.970264e+15,3.376970e+15,4.523700e+15,3.469726e+15,2.401444e+15,3.427217e+15,3.243274e+15,3.390173e+15,2.871262e+15,5.052352e+15,5.412090e+15,3.342013e+15,2.168255e+15,5.595358e+15,4.130868e+15,3.344739e+15,3.179874e+15,7.042156e+15,4.192819e+15,2.169621e+15,4.228303e+15,5.811587e+15,2.312069e+15,3.833379e+15,1.701940e+15,3.363910e+15,4.468970e+15,5.285087e+15,5.611119e+15,5.526571e+15,4.502730e+15
4,5.022438e+15,4.857102e+15,4.901760e+15,5.067914e+15,4.832015e+15,4.866162e+15,5.021683e+15,4.962558e+15,5.088225e+15,5.230106e+15,4.156916e+15,3.535730e+15,4.736371e+15,3.632846e+15,2.514341e+15,3.588339e+15,3.395749e+15,3.549554e+15,3.006247e+15,5.289876e+15,5.666526e+15,3.499129e+15,2.270191e+15,5.858410e+15,4.325070e+15,3.501984e+15,3.329368e+15,7.373225e+15,4.389934e+15,2.271620e+15,4.427086e+15,6.084805e+15,2.420765e+15,4.013596e+15,1.781953e+15,3.522056e+15,4.679067e+15,5.533552e+15,5.874912e+15,5.786389e+15,4.714415e+15
5,5.087038e+15,4.919576e+15,4.964807e+15,5.133099e+15,4.894165e+15,4.928752e+15,5.086274e+15,5.026388e+15,5.153670e+15,5.297376e+15,4.210384e+15,3.581207e+15,4.797291e+15,3.679573e+15,2.546682e+15,3.634493e+15,3.439425e+15,3.595209e+15,3.044914e+15,5.357915e+15,5.739410e+15,3.544136e+15,2.299390e+15,5.933762e+15,4.380700e+15,3.547027e+15,3.372191e+15,7.468061e+15,4.446398e+15,2.300839e+15,4.484028e+15,6.163069e+15,2.451902e+15,4.065220e+15,1.804873e+15,3.567357e+15,4.739250e+15,5.604726e+15,5.950476e+15,5.860815e+15,4.775053e+15
6,4.761120e+15,4.604387e+15,4.646721e+15,4.804230e+15,4.580605e+15,4.612976e+15,4.760405e+15,4.704356e+15,4.823484e+15,4.957983e+15,3.940632e+15,3.351765e+15,4.489937e+15,3.443829e+15,2.383520e+15,3.401638e+15,3.219068e+15,3.364870e+15,2.849832e+15,5.014643e+15,5.371696e+15,3.317069e+15,2.152072e+15,5.553596e+15,4.100036e+15,3.319775e+15,3.156140e+15,6.989596e+15,4.161525e+15,2.153428e+15,4.196744e+15,5.768212e+15,2.294813e+15,3.804768e+15,1.689238e+15,3.338803e+15,4.435615e+15,5.245641e+15,5.569240e+15,5.485323e+15,4.469124e+15
7,4.878645e+15,4.718043e+15,4.761422e+15,4.922819e+15,4.693674e+15,4.726844e+15,4.877912e+15,4.820479e+15,4.942548e+15,5.080367e+15,4.037903e

In [267]:
s

i
0       250.06
1       254.73
2       253.17
3       246.53
4       258.12
5       261.44
6       244.69
7       250.73
8       238.79
9       253.33
AT      744.00
BR     1191.36
CA     1948.38
CH     1307.46
CN     9201.96
CZ      703.38
DE     6380.94
DK      479.58
DZ      339.00
ES     1276.80
GB     2052.06
HU      474.48
IE      659.70
IN     1242.96
IT     2304.66
JP     3671.88
KR     2493.72
MA       95.04
NL     2343.24
NO      722.82
PL      833.52
PT      268.80
RU     2250.54
RW    18837.12
SA     1394.88
SE      837.66
SG     1143.90
TN       87.30
TR      653.28
US     6038.64
XU     1780.56
dtype: float64